In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import os
import re
import sys
from datetime import datetime
import memory_profiler
import gc
from multiprocessing import Process
import multiprocessing as mp

sys.path.insert(0, '../../drlib')
import drlib as dr

%matplotlib widget

In [29]:
dataDir = '/drBiggerBoy/drData/Data/'
fileList = os.listdir(dataDir)
fileList.sort(key=lambda f: int(re.sub('\D', '', f)))

fileName = 'preProcDataSet.hdf5'

In [30]:
header = ['File', 'Date', 'Temperture', 'West', 'Vertical', 'South', 'Phi', 'Theta']
database = pd.read_csv('databaseForPandas.txt', names=header, skiprows=1)
database['Date'] = pd.to_datetime(database['Date'], infer_datetime_format=True)

database.set_index('Date', inplace=True)
database.drop(columns='File', inplace=True)
print(database.head())


                         Temperture  West  Vertical  South  Phi  Theta
Date                                                                  
2022-07-07 12:46:11.184      294.35   140       104    200    0     45
2022-07-07 12:47:09.099      294.35   140       104    200    0     45
2022-07-07 12:48:07.022      294.35   140       104    200    0     45
2022-07-07 12:49:04.936      294.35   140       104    200    0     45
2022-07-07 12:50:02.850      294.35   140       104    200    0     45


In [47]:
#pack new h5 file 
f.close()
f = h5py.File('/drBiggerBoy/drData/run1Data/' + fileName, 'w')


plt.close('all')

numFiles = 0
numMeasData = 0
numRigolSpec = 0
rigolQueueArr = np.empty((10000, 4))


def packRoachDataAndAttrs(file, measData, key):
    dataset = pd.read_hdf(dataDir + file, key = measData)
    dateTimeStr = dataset.columns[0][0]
    dateTime = datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    #print(database.loc[dateTime])
    diffSpec = (dr.fft2Watts(np.asarray(dataset.iloc[:,2][1:])))

    grp = f.create_group(dateTimeStr)
    

    header = ['Temperture', 'West', 'Vertical', 'South', 'Phi', 'Theta']
    for attr in header:
        grp.attrs.create(attr, database.loc[dateTime][attr])
    dset = grp.create_dataset('diffSpecWatts', data = diffSpec)

def packRigolData(file, measData, key):
    rigolIdx = 0
    rigolQueueArr[:,:] = 0

    rigolQueueArr[:, rigolIdx] = []
    rigolIdx += 1


########################################
#Main loop
########################################
for file in fileList[0:1]:
    dataset = h5py.File(dataDir +  file, 'r')
    numFiles += 1
    measDataKeys = list(dataset.keys())
    measDataKeys.sort(key=lambda f: int(re.sub('\D', '', f)))

    for measData in measDataKeys:
        #print(measData)
        #dataset = pd.read_hdf(dataDir + file, key = measData)
        #dataset.head()
        measDataSubKeys = dataset[measData].keys()
        print(measDataSubKeys)
        numMeasData += 1
    
        #Pack ROACH diff spec
        #print(numMeasData)
        #dataTemp = (dataset[measData]['table'][:])
        #diffSpec = [x[1][2] for x in dataTemp]
        #print(diffSpec)
        #make array of rigol veto max values
        rigolQueueArr[:,:] = 0
        rigolIdx = 0
        for key in measDataSubKeys:
            #packRoachData(file, measData, key)
            if key=='table':
                packRoachDataAndAttrs(file, measData, key)

            if key[0] == 'R':
                #print('key = ', key)
                packRigolData(file, measData, key)
                rigolQueueArr[:, rigolIdx] = []
                rigolIdx += 1
         

                #timeStamp = datetime.strptime(h5py.File(dataDir +  file, 'r')[measData][key].attrs['time'], '%Y-%m-%d %H:%M:%S.%f')
                #print(timeStamp)
                #rigolDfTemp[timeStamp] = h5py.File(dataDir +  file, 'r')[measData][key]

                #numRigolSpec += 1
            
print('number of files = ', numFiles)
print('number of meas data = ', numMeasData)
#print('number of Rigol spectra = ', numRigolSpec)
#print('df size: ', sys.getsizeof(rigolDf)/1e6, 'MB')
f.close()

<KeysViewHDF5 ['RIGOL_0', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
<KeysViewHDF5 ['RIGOL_0', 'RIGOL_1', '_i_table', 'table']>
number of files =  1
number of meas data =  16


In [26]:
f = h5py.File('h5Test.hdf5', 'w')
f.close()
fileName = 'h5Test.hdf5'


def worker(fileName, num):
    
    f = h5py.File(fileName, 'w')
    dset = f.create_dataset(str(num), data = 69)
    grp = f.create_group('asdf')
    
    f.close()

'''
if __name__ == '__main__':
    output = mp.Queue()
    inqueue = mp.Queue()
    jobs = []
    proc = mp.Process(target=handle_output, args=(output, ))
    proc.start()
    for i in range(num_processes):
        p = mp.Process(target=Simulation, args=(inqueue, output))
        jobs.append(p)
        p.start()
    for i in range(num_simulations):
        inqueue.put(i)
    for i in range(num_processes):
        # Send the sentinal to tell Simulation to end
        inqueue.put(sentinel)
    for p in jobs:
        p.join()
    output.put(None)
    proc.join()
'''


for num in range(10):
        Process(target=worker, args= (fileName, num,)).start()
        #print(num)

f.close()

In [11]:
f.close()

In [ ]:
f.close()